 # Homework # 4 - Random Forest
 Data file: temps_extended.csv

### Import libraries

In [21]:
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

### Function to evaluate model accuracy
#### You will use this function twice in this notebook.
Please review the comments below carefully to find out when to invoke this function.

In [22]:
def evaluate(model, model_string, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance: {}'.format(model_string))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

### Load data

In [23]:
# Read in data as a dataframe
temps_extended = pd.read_csv('data/temps_extended.csv')

### Examine data

In [24]:
temps_extended.shape

(2191, 12)

(2191, 12)

In [25]:
temps_extended.head(10)

,year,month,day,weekday,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,2011,1,1,Sat,4.92,0.00,0,36,37,45.6,40,40
1,2011,1,2,Sun,5.37,0.00,0,37,40,45.7,39,50
2,2011,1,3,Mon,6.26,0.00,0,40,39,45.8,42,42
3,2011,1,4,Tues,5.59,0.00,0,39,42,45.9,38,59
4,2011,1,5,Wed,3.80,0.03,0,42,38,46.0,45,39
5,2011,1,6,Thurs,4.92,0.12,0,38,45,46.1,49,52
6,2011,1,7,Fri,8.72,0.17,0,45,49,46.2,48,62
7,2011,1,8,Sat,10.96,0.37,0,49,48,46.3,40,65
8,2011,1,9,Sun,5.82,0.03,0,48,40,46.4,39,53
9,2011,1,10,Mon,5.59,0.02,0,40,39,46.5,38,40


,year,month,day,weekday,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,2011,1,1,Sat,4.92,0.00,0,36,37,45.6,40,40
1,2011,1,2,Sun,5.37,0.00,0,37,40,45.7,39,50
2,2011,1,3,Mon,6.26,0.00,0,40,39,45.8,42,42
3,2011,1,4,Tues,5.59,0.00,0,39,42,45.9,38,59
4,2011,1,5,Wed,3.80,0.03,0,42,38,46.0,45,39
5,2011,1,6,Thurs,4.92,0.12,0,38,45,46.1,49,52
6,2011,1,7,Fri,8.72,0.17,0,45,49,46.2,48,62
7,2011,1,8,Sat,10.96,0.37,0,49,48,46.3,40,65
8,2011,1,9,Sun,5.82,0.03,0,48,40,46.4,39,53
9,2011,1,10,Mon,5.59,0.02,0,40,39,46.5,38,40


### Clean up data

In [26]:
# Drop unnecessary columns year, month, day, weekday
temps_extended = temps_extended.drop("year", axis = 1)
temps_extended = temps_extended.drop("month", axis = 1)
temps_extended = temps_extended.drop("day", axis = 1)
temps_extended = temps_extended.drop("weekday", axis = 1)

In [27]:
# Display first few rows of updated dataframe
temps_extended.head(10)

,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,4.92,0.00,0,36,37,45.6,40,40
1,5.37,0.00,0,37,40,45.7,39,50
2,6.26,0.00,0,40,39,45.8,42,42
3,5.59,0.00,0,39,42,45.9,38,59
4,3.80,0.03,0,42,38,46.0,45,39
5,4.92,0.12,0,38,45,46.1,49,52
6,8.72,0.17,0,45,49,46.2,48,62
7,10.96,0.37,0,49,48,46.3,40,65
8,5.82,0.03,0,48,40,46.4,39,53
9,5.59,0.02,0,40,39,46.5,38,40


,ws_1,prcp_1,snwd_1,temp_2,temp_1,average,actual,friend
0,4.92,0.00,0,36,37,45.6,40,40
1,5.37,0.00,0,37,40,45.7,39,50
2,6.26,0.00,0,40,39,45.8,42,42
3,5.59,0.00,0,39,42,45.9,38,59
4,3.80,0.03,0,42,38,46.0,45,39
5,4.92,0.12,0,38,45,46.1,49,52
6,8.72,0.17,0,45,49,46.2,48,62
7,10.96,0.37,0,49,48,46.3,40,65
8,5.82,0.03,0,48,40,46.4,39,53
9,5.59,0.02,0,40,39,46.5,38,40


### Separate independent variables and dependent variable
* Independent variables: all remaining variables except 'actual'
* Dependent variable: 'actual'

In [28]:
# Separate features and labels
X = temps_extended.drop('actual', axis = 1)
y = temps_extended['actual']

### Split into training and test sets

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Instantiate the RandomForestRegressor model

In [30]:
rf = RandomForestRegressor(n_estimators= 100, random_state=42)

### Print RandomForestRegressor default hyperparameters

In [31]:
print('RandomForestRegressor default hyperparameters\n')
pprint(rf.get_params())

RandomForestRegressor default hyperparameters

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}
RandomForestRegressor default hyperparameters

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


### Fit RandomForestRegressor model using the default hyperparameters

In [32]:
%%time

rf.fit(X_train, y_train);

Wall time: 585 ms
Wall time: 585 ms


RandomForestRegressor(random_state=42)

RandomForestRegressor(random_state=42)

### Print accuracy for RandomForestRegressor model using the default hyperparameters
#### NOTE: Use "evaluate" function defined at top of this notebook.
For example, assuming the following variable values:
* model = rf
* model_string = 'using default hyperparameters'
* test_features = X_test
* test_labels = y_test

rfr_base_accuracy = evaluate(rf, 'With default hyperparameters', X_test, y_test)

In [33]:
rfr_base_accuracy = evaluate(rf, 'With default hyperparameters', X_test, y_test)

Model Performance: With default hyperparameters
Accuracy = 93.44%.
Model Performance: With default hyperparameters
Accuracy = 93.44%.


### Prepare variables for hyperparameter search
* Using sklearn.ensemble.RandomForestRegressor documentation [https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html] choose 3 hyperparameters for random search
* For each hyperparameter selected, set up an array of values
  * For example: max_features = ['log2', 'sqrt']

In [34]:
n_estimators = np.arange(50, 100, 10),
max_depth = [int(x) for x in np.linspace(2, 20, num = 10)],
min_samples_split = [3,6,9,12,15]

### Create the hyperparameter grid for the random search
Use the variables prepared above

In [35]:
grid = {'n_estimators': np.arange(50, 100, 10),
'max_depth': np.arange(10, 50, 3),
'min_samples_split': [3,6,9,12,15]}

### Print the hyperparameter grid for the random search

In [36]:
print(grid)

{'n_estimators': array([50, 60, 70, 80, 90]), 'max_depth': array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49]), 'min_samples_split': [3, 6, 9, 12, 15]}
{'n_estimators': array([50, 60, 70, 80, 90]), 'max_depth': array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49]), 'min_samples_split': [3, 6, 9, 12, 15]}


### Set up random search with k-fold cross validation using the hyperparameter grid

In [37]:
# Use RandomizedSearchCV to perform random search of hyperparameters using 5 fold cross validation.
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)

### Fit the random search model
Be patient, this might take a minute or longer

In [38]:
%%time
rf_random.fit(X_train, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Wall time: 4.57 s
Wall time: 4.57 s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49]),
                                        'min_samples_split': [3, 6, 9, 12, 15],
                                        'n_estimators': array([50, 60, 70, 80, 90])},
                   random_state=42, verbose=2)

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': array([10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49]),
                                        'min_samples_split': [3, 6, 9, 12, 15],
                                        'n_estimators': array([50, 60, 70, 80, 90])},
                   random_state=42, verbose=2)

### Print the best hyperparameters found by the random search

In [39]:
print("Best hyperparameters found by random search:")
pprint(rf_random.best_params_)

Best hyperparameters found by random search:
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 90}
Best hyperparameters found by random search:
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 90}


### Print best random search model accuracy
#### NOTE: Use "evaluate" function defined at top of this notebook.

In [40]:
rf_random.best_estimator_.fit(X_train, y_train)
rf_random = evaluate(rf_random, 'With random hyperparameters', X_test, y_test)

Model Performance: With random hyperparameters
Accuracy = 93.62%.
Model Performance: With random hyperparameters
Accuracy = 93.62%.
